In [1]:
import os
os.getcwd()

'/teamspace/studios/this_studio/text-summarizer/research'

In [2]:
os.chdir('../')
os.getcwd()

'/teamspace/studios/this_studio/text-summarizer'

In [3]:
from rich import print

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: str
    output_dir: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    logging_dir: Path
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int

In [5]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath= CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        os.makedirs(self.config.artifact_root, exist_ok=True)
        
        # print("Config: ", self.config)
        
    def get_model_trainer_config(self)-> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments
        
        os.makedirs(config.root_dir, exist_ok=True)
        print(f"Config from Configmanager: {config}")
        
        return ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            output_dir = params.output_dir,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            per_device_eval_batch_size = params.per_device_eval_batch_size,
            logging_dir = params.logging_dir,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

In [7]:
# !pip install --upgrade pip
# !pip uninstall torchvision -qy
# !pip install torchvision -q
# !pip install -U transformers accelerate -q

In [8]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from datasets import load_from_disk

2.6.0+cu124

0.21.0+cu124

In [12]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        print(f"Config from ModelTrainer: {config}")
    def start_training(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        # print(self.config.model_ckpt)
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt)
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
        
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        training_args = TrainingArguments(
            output_dir = self.config.output_dir,
            num_train_epochs = self.config.num_train_epochs,
            per_device_train_batch_size = self.config.per_device_train_batch_size,
            per_device_eval_batch_size = self.config.per_device_eval_batch_size,
            logging_dir = self.config.logging_dir,
            logging_steps = self.config.logging_steps,
            evaluation_strategy = self.config.evaluation_strategy,
            eval_steps = self.config.eval_steps,
            save_steps = self.config.save_steps,
            gradient_accumulation_steps = self.config.gradient_accumulation_steps,
            warmup_steps = self.config.warmup_steps,
            weight_decay = self.config.weight_decay
        )
        
        trainer = Trainer(
            model = model,
            args = training_args,
            data_collator = data_collator,
            train_dataset = dataset_samsum_pt['test'],
            eval_dataset = dataset_samsum_pt['validation']
        )
        
        trainer.train()
        
        model.save_pretrained(os.path.join(self.config.root_dir, "trained-pegausus-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "trained-pegausus-tokenizer"))

In [13]:
%%time
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
# print(f"Model Trainer Config: {model_trainer_config}")
model_trainer = ModelTrainer(model_trainer_config)
model_trainer.start_training()

Config from Configmanager: {'root_dir': 'artifacts/model_trainer', 'data_path': 
'artifacts/data_transformation/samsum_dataset', 'model_ckpt': 'google/pegasus-cnn_dailymail'}

Config from ModelTrainer: ModelTrainerConfig(root_dir='artifacts/model_trainer', 
data_path='artifacts/data_transformation/samsum_dataset', model_ckpt='google/pegasus-cnn_dailymail', 
output_dir='pegasus_samsum', num_train_epochs=2, warmup_steps=500, per_device_train_batch_size=1, 
per_device_eval_batch_size=1, logging_dir='./logs', weight_decay=0.01, logging_steps=10, 
evaluation_strategy='steps', eval_steps=500, save_steps=999, gradient_accumulation_steps=16)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
